In [3]:
# Make sure to install youtube-dl (go to website) and ffmpeg (brew install ffmpeg)
import csv, sys
import os
import wave
import contextlib
import pandas as pd
from multiprocessing import Pool

In [4]:
# Download unbalanced_train_segments.csv, and replace directory with yours.
filename = 'Data/unbalanced_train_segments.csv'
path = 'Data/unbalanced_train_wav/'
project_path = '/Users/mingukim/Documents/cs281_fp/AudioSet_GAN/'
error_path = 'Data/unbalanced_train_wav/error_wav/'
unbal_train = pd.read_csv(filename, names=['tag', 'start', 'end', 'label1', 'label2', 'label3', 'label4', 'label5', 'label6', 'label7', 'label8', 'label9', 'label10','label11','label12'])

/Library/Frameworks/Python.framework/Versions/3.4/lib/python3.4/site-packages/IPython/core/interactiveshell.py:2705: DtypeWarning: Columns (1,2,13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
# We want to only select human voice related sound files- check indices file to see how many labels you want to include.
indices = pd.read_csv('Data/class_labels_indices.csv')
acc_cl = indices['mid'][:6]
acc_cl = [' "' + x for x in acc_cl]

In [6]:
''' We'd like to thank and acknowledge these functions provided by DantesLegacy as an example of how to 
    extract Youtube videos (https://github.com/DantesLegacy/TensorFlow_AudioSet_Example). 
'''
def youtube_download_os_call(id, start_time, idx):
    ret = os.system('ffmpeg -n -ss ' + start_time +
              ' -i $(youtube-dl -i -w --extract-audio '
              '--audio-format wav --audio-quality 0 '
              '--get-url https://www.youtube.com/watch?v=' + id + ')'
              ' -t 10 ' + path + idx + '_' + id + '.wav')

    return ret

def get_wav_file_length(path, idx, id):
    sample = project_path + path + idx + '_' + id + '.wav'
    with contextlib.closing(wave.open(sample, 'r')) as f:
        frames = f.getnframes()
        rate = f.getframerate()
        length = frames / float(rate)
        print(length)

    return length

def create_error_file(id, idx):
    with open(error_path + idx + '_' + id + '_ERROR.wav', 'a'):
        os.utime(error_path + idx + '_' + id + '_ERROR.wav', None)

def youtube_downloader(id, start_time, idx):
    ret = youtube_download_os_call(id, start_time, idx)

    print('ffmpeg -n -ss ' + start_time +
              ' -i $(youtube-dl -i -w --extract-audio '
              '--audio-format wav --audio-quality 0 '
              '--get-url https://www.youtube.com/watch?v=' + id + ')'
              ' -t 10 AudioSet/balanced_train/' + idx + '_' + id + '.wav')
    return ret

In [16]:
# Download .wav files. Some files may not download, because of videos that have been taken down.
# N = number of videos considered, not final number of .wav files (which depends on number of errors)
N = 20

# Take only voice related videos
voice_data = unbal_train.loc[unbal_train['label1'].isin(acc_cl)][:N]

In [17]:
%%time

#Download the .wav files
for index, clip in voice_data.iterrows():
    ret = youtube_downloader(clip[0], str(clip[1]), str(index))
    if ret != 0:
        create_error_file(clip[0], str(index))
        
    

ffmpeg -n -ss  30.000 -i $(youtube-dl -i -w --extract-audio --audio-format wav --audio-quality 0 --get-url https://www.youtube.com/watch?v=---EDNidJUA) -t 10 AudioSet/balanced_train/6_---EDNidJUA.wav
ffmpeg -n -ss  30.000 -i $(youtube-dl -i -w --extract-audio --audio-format wav --audio-quality 0 --get-url https://www.youtube.com/watch?v=--1l6WGE7AU) -t 10 AudioSet/balanced_train/41_--1l6WGE7AU.wav
ffmpeg -n -ss  10.000 -i $(youtube-dl -i -w --extract-audio --audio-format wav --audio-quality 0 --get-url https://www.youtube.com/watch?v=--2U9LPjlpk) -t 10 AudioSet/balanced_train/55_--2U9LPjlpk.wav
ffmpeg -n -ss  520.000 -i $(youtube-dl -i -w --extract-audio --audio-format wav --audio-quality 0 --get-url https://www.youtube.com/watch?v=--3SMQlk4Y8) -t 10 AudioSet/balanced_train/74_--3SMQlk4Y8.wav
ffmpeg -n -ss  30.000 -i $(youtube-dl -i -w --extract-audio --audio-format wav --audio-quality 0 --get-url https://www.youtube.com/watch?v=--42a6bv16w) -t 10 AudioSet/balanced_train/82_--42a6bv16w

In [18]:
%%time

def clip_worker(arg):
    index, clip = arg
    ret = youtube_downloader(clip[0], str(clip[1]), str(index))
    if ret != 0:
        create_error_file(clip[0], str(index))
        
with Pool(5) as p:
    p.map(clip_worker, [(index, clip) for index, clip in voice_data.iterrows()])

ffmpeg -n -ss  30.000 -i $(youtube-dl -i -w --extract-audio --audio-format wav --audio-quality 0 --get-url https://www.youtube.com/watch?v=--1l6WGE7AU) -t 10 AudioSet/balanced_train/41_--1l6WGE7AU.wav
ffmpeg -n -ss  520.000 -i $(youtube-dl -i -w --extract-audio --audio-format wav --audio-quality 0 --get-url https://www.youtube.com/watch?v=--3SMQlk4Y8) -t 10 AudioSet/balanced_train/74_--3SMQlk4Y8.wav
ffmpeg -n -ss  30.000 -i $(youtube-dl -i -w --extract-audio --audio-format wav --audio-quality 0 --get-url https://www.youtube.com/watch?v=---EDNidJUA) -t 10 AudioSet/balanced_train/6_---EDNidJUA.wav
ffmpeg -n -ss  30.000 -i $(youtube-dl -i -w --extract-audio --audio-format wav --audio-quality 0 --get-url https://www.youtube.com/watch?v=--42a6bv16w) -t 10 AudioSet/balanced_train/82_--42a6bv16w.wav
ffmpeg -n -ss  10.000 -i $(youtube-dl -i -w --extract-audio --audio-format wav --audio-quality 0 --get-url https://www.youtube.com/watch?v=--2U9LPjlpk) -t 10 AudioSet/balanced_train/55_--2U9LPjlpk